In [18]:
import os
from IPython.display import Image as ipy_image
from IPython.display import display

#from utils.demo_util import get_shape_comp_model, get_shape_comp_opt, make_dummy_batch
#from utils.qual_util import load_bert2vqsc_model, get_lang_prob, save_mesh_as_gif
from utils.qual_util import get_lang_prob, save_mesh_as_gif
from utils.util import seed_everything
from utils.util_3d import init_mesh_renderer, sdf_to_mesh
import os
from termcolor import colored, cprint
import torch
import utils.util as util
from tqdm.notebook import tqdm

from datasets.ys_shapeset import  ShapeNetZSets


%reload_ext autoreload
%autoreload 2

In [28]:
seed_everything(111)

description = "Loading new bert mode no probs passed"
res_dir = 'logs/comparison/v14'
if not os.path.exists(res_dir): os.makedirs(res_dir)

with open(f"{res_dir}/description.txt","w") as f:
    f.write(description)

gpu_id = 0
nimgs=6

class Opt:
    def __init__(self):
        self.name = 'opt'

In [20]:
def load_bert2vqsc_model(opt):
    from models.networks.bert2vq_sc import BERT2VQ
    
    net = BERT2VQ(opt)
    # bert2vq_ckpt = '/home/paritosh/Desktop/Capstone/clean-code/generative_transformers/logs/bert2vq-shapenet_lang-all-LR1e-4-cleanCode-langMode-/ckpt/bert2vq_epoch-145.pth'
    bert2vq_ckpt = '../raw_dataset/checkpoints_new/bert2vq_epoch-latest.pth'
    state_dict = torch.load(bert2vq_ckpt)
    net.load_state_dict(state_dict['bert2vq'])
    net.eval()
    net.to(opt.device)
    
    return net

In [4]:
shapeset = ShapeNetZSets()

In [8]:
y  = shapeset[4]["z_set"]
y.shape


torch.Size([8, 8, 8, 512])

In [21]:
def create_model(opt):
    model = None

    if opt.model == 'pvqvae':
        # vqvae
        from models.pvqvae_model import PVQVAEModel
        model = PVQVAEModel()
    elif opt.model == 'rand_tf':
        # transformer
        from models.rand_tf_model import RandTransformerModel
        model = RandTransformerModel()
    elif opt.model == 'rand_tf_old':
        from models.rand_tf_model_old import RandTransformerModelOld
        model = RandTransformerModelOld()
    elif opt.model == 'seq_tf':
        # seq-transformer
        from models.seq_tf_model import SeqTransformerModel
        model = SeqTransformerModel()
    elif opt.model == 'bert2vq':
        from models.bert2vq_model import BERT2VQModel
        model = BERT2VQModel()
    elif opt.model == "bert2vqsc":
        from models.bert2vq_scmodel import BERT2VQSCModel
        model = BERT2VQSCModel()
    elif opt.model == 'resnet2vq':
        from models.resnet2vq_model import ResNet2VQModel
        model = ResNet2VQModel()
    elif opt.model == 'resnet2vox':
        from models.resnet2vox_model import ResNet2VoxModel
        model = ResNet2VoxModel()
    elif opt.model == 'resnet2sdf':
        from models.resnet2sdf_model import ResNet2SDFModel
        model = ResNet2SDFModel()
    elif opt.model == 'baseline_je':
        from models.baseline_je_model import LangJEModel
        model = LangJEModel()
    elif opt.model == 'img_je':
        from models.img_je_model import ImgJEModel
        model = ImgJEModel()
    else:
        raise ValueError("Model [%s] not recognized." % opt.model)

    model.initialize(opt)
    cprint("[*] Model has been created: %s" % model.name(), 'blue')
    if(opt.ckpt is not None):
        model.load_ckpt(opt.ckpt)
        print("MODEL LOADED")
    return model

In [22]:
def get_shape_comp_opt(gpu_id=0):
    opt = Opt()

    # args
    gpuid=[gpu_id]
    batch_size=1
    max_dataset_size=10000000

    name='test_transformer'

    # default args
    opt.serial_batches = False
    opt.nThreads = 4

    # important args
    opt.dataset_mode = 'shapenet_code'
    opt.seed = 111
    opt.isTrain = False
    opt.gpu_ids = gpuid
    opt.device = 'cuda:%s' % gpuid[0]
    opt.batch_size = batch_size
    opt.max_dataset_size = max_dataset_size

    opt.name = name

    #utils.util.seed_everything(opt.seed)
    opt.phase = 'test'
    return opt




def get_shape_comp_model(opt, model, ckpt):
    
    # load tf stuff
    opt.model=model
    opt.tf_cfg='configs/rand_tf_snet_code.yaml'
    opt.ckpt = ckpt
    
    # load vq stuff
    opt.vq_model='pvqvae'
    opt.vq_cfg='configs/pvqvae_snet.yaml'
    opt.vq_ckpt='../raw_dataset/checkpoints/vqvae.pth'
    
    ### opt.vq_dset='sdf_code' # original
    opt.vq_dset='snet'

    model = create_model(opt)
    print(f'[*] "{opt.model}" initialized.')
    model.load_ckpt(opt.ckpt)
        
    return model

In [9]:
opt = get_shape_comp_opt(gpu_id=gpu_id)
opt.dataset_mode = "text2shape"
modelOld = get_shape_comp_model(opt,"rand_tf_old","../raw_dataset/checkpoints/rand_tf_singles_best.pth")    
modelOld.eval()

#new_checkpoint_path = "logs/valid-valid-with-fusion-rand_tf-owndataset-chair-LR1e-4-clean-with-ckpt/ckpt/rand_tf_epoch-latest.pth"
new_checkpoint_path = "../raw_dataset/checkpoints_new/rand_tf_epoch_latest_fusion_transfer_v1.pth"
modelNew = get_shape_comp_model(opt,"rand_tf",new_checkpoint_path) 
modelNew.eval()



[*] Enc has Attn at i_level, i_block: 3, 0
Working with z of shape (1, 256, 8, 8, 8) = 131072 dimensions.
[*] Dec has Attn at i_level, i_block: 3, 0
[*] VQVAE: weight successfully load from: ../raw_dataset/checkpoints/vqvae.pth


/rhome/streakfull/.cache/pypoetry/virtualenvs/adl4cv-I-Koul65-py3.8/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[*] Model has been created: Rand-Transformer-Model
[*] weight successfully load from: ../raw_dataset/checkpoints/rand_tf_singles_best.pth
MODEL LOADED
[*] "rand_tf_old" initialized.
[*] weight successfully load from: ../raw_dataset/checkpoints/rand_tf_singles_best.pth
[*] Enc has Attn at i_level, i_block: 3, 0
Working with z of shape (1, 256, 8, 8, 8) = 131072 dimensions.
[*] Dec has Attn at i_level, i_block: 3, 0
[*] VQVAE: weight successfully load from: ../raw_dataset/checkpoints/vqvae.pth
[*] Model has been created: Rand-Transformer-Model
[*] weight successfully load from: ../raw_dataset/checkpoints_new/rand_tf_epoch_latest_fusion_transfer_v1.pth
MODEL LOADED
[*] "rand_tf" initialized.
[*] weight successfully load from: ../raw_dataset/checkpoints_new/rand_tf_epoch_latest_fusion_transfer_v1.pth


In [24]:
""" setup renderer """
dist, elev, azim = 1.7, 20, 20
mesh_renderer = init_mesh_renderer(image_size=256, dist=dist, elev=elev, azim=azim, device=opt.device)

opt.mlp_layers = 3
opt.mlp_hidden = 1024
bert2vq = load_bert2vqsc_model(opt)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
with open("./test_samples_paper.txt") as file:
    sequences = [line.rstrip() for line in file]




In [26]:
sequences_clean = [0] * len(sequences)
for i in range(len(sequences)):
    seq = sequences[i]
    sequence = seq.split(",")
    sequences_clean[i] = sequence

In [27]:
sequences_clean[0]

['a high bar seat',
 'has a molded curved seat',
 'has a metal footrest and circular base']

In [12]:
model = modelOld
model_name = "model_old"

import torch
from utils.qual_util import get_lang_prob_recursive, save_mesh_as_pics
from einops import rearrange

z1 = torch.full((1,8,8,8,512), 1/512).cuda()

test_data = { }
sampler = torch.distributions.categorical.Categorical(z1)
codeix = sampler.sample()
test_data["code"] = z1.repeat_interleave(nimgs, dim=0)
test_data["idx"] = sampler.sample().repeat_interleave(nimgs, axis=0).cpu()
test_data["z_q"] = z1


for sequence in sequences_clean:
    last_text = ""
    for text in sequence:
        all_text = last_text + text
        text_conditional = text
        lang_conditional_prob = get_lang_prob_recursive(bert2vq, text_conditional, z1)
        z1_old = rearrange(z1, 'bs d h w c -> (d h w bs) c')
        z1 = rearrange(lang_conditional_prob, '(d h w) bs c -> bs d h w c', d=8, h=8, w=8)
        gen_order = torch.argsort((torch.abs(lang_conditional_prob[:, 0, :]-z1_old)).sum(-1), dim=-1, descending=True)
        lang_conditional_prob = lang_conditional_prob.repeat(1, nimgs, 1)
        topk = 50
        model.inference(test_data, topk=topk, prob=None,gen_order=None)
        test_data["code"] = z1.repeat_interleave(nimgs, dim=0)
        sampler = torch.distributions.categorical.Categorical(z1)
        test_data["idx"] = torch.cat( [sampler.sample().cpu() for i in range(6)], dim=0)
        print(test_data["idx"].shape)
        gen_mesh = sdf_to_mesh(model.x_recon_tf)
    
        gen_gif_name = f'{res_dir}/{model_name}-{all_text}.gif'
        #import pdb;pdb.set_trace()
        try:
            save_mesh_as_pics(mesh_renderer, gen_mesh, nrow=3, out_name=gen_gif_name)
        except:
            import pdb;pdb.set_trace()
        last_text = text_conditional + " "
        
        print(text_conditional)
        for name in [gen_gif_name]:
            display(ipy_image(name))
        

[*] autoregressively inferencing...:  63%|█████████████████████████████████████████████████▉                             | 324/512 [00:02<00:01, 125.69it/s]


KeyboardInterrupt: 

In [ ]:
torch.randperm(512, device='cuda').shape

In [32]:
model = modelNew
model_name = "model_new"

import torch
from utils.qual_util import get_lang_prob_recursive, save_mesh_as_pics
from einops import rearrange




for j in tqdm(range(len(sequences_clean))):
    #text = sequence[j]
    sequence = sequences_clean[j]
    last_text = ""
    z1 = torch.full((1,8,8,8,512), 1/512).cuda()
    test_data = { }
    sampler = torch.distributions.categorical.Categorical(z1)
    codeix = sampler.sample()
    
    
    #test_data["z_set"] = z1.repeat_interleave(nimgs, dim=0)
    test_data["z_set"] = shapeset[4]["z_set"].unsqueeze(0).repeat_interleave(nimgs, dim=0)
   
    test_data["idx"] = sampler.sample().repeat_interleave(nimgs, axis=0).cpu()
    test_data["z_q"] = z1
    print("======================================================================================================")
    for i in range(len(sequence)):
        text = sequence[i]
        all_text = last_text + text
        text_conditional = text
        lang_conditional_prob = get_lang_prob_recursive(bert2vq, all_text, z1)
        z1_old = rearrange(z1, 'bs d h w c -> (d h w bs) c')
        z1 = rearrange(lang_conditional_prob, '(d h w) bs c -> bs d h w c', d=8, h=8, w=8)
        gen_order = torch.argsort((torch.abs(lang_conditional_prob[:, 0, :]-z1_old)).sum(-1), dim=-1, descending=True)
        lang_conditional_prob = lang_conditional_prob.repeat(1, nimgs, 1)
        topk = 50
        # This should be replaced by bert output
        #test_data["z_set"] = z1.repeat_interleave(nimgs, dim=0)
        model.inference(test_data, topk=topk, prob=None,gen_order=None)
        #test_data["z_set"] = z1.repeat_interleave(nimgs, dim=0)
        sampler = torch.distributions.categorical.Categorical(z1)
        test_data["idx"] = torch.cat( [sampler.sample().cpu() for i in range(6)], dim=0)
        print(test_data["idx"].shape)
        gen_mesh = sdf_to_mesh(model.x_recon_tf)
    
        gen_gif_name = f'{res_dir}/{j}-{i}-{model_name}-{all_text}.gif'
        #import pdb;pdb.set_trace()
        try:
            save_mesh_as_pics(mesh_renderer, gen_mesh, nrow=3, out_name=gen_gif_name)
        except:
            import pdb;pdb.set_trace()
        last_text = text_conditional + " "
        
        print("TEXT:", text_conditional)
        print("Full Text:", all_text)
        print("Saved:", gen_gif_name)
        for name in [gen_gif_name]:
            display(ipy_image(name))

  0%|          | 0/13 [00:00<?, ?it/s]

RuntimeError: number of dims don't match in permute